
# 05. 딥러닝 기반 접근: FiBiNet 구현

본 노트북은 DeepCTR 계열의 **FiBiNet (Feature Importance and Bilinear Interaction Network)** 모델을 구현합니다.
기존 트리 기반 모델(GBDT)이 포착하기 어려운 피처 간의 미세한 상호작용과 중요도를 딥러닝 네트워크로 학습합니다.

### 💡 핵심 아키텍처
1. **SENet (Squeeze-and-Excitation Network)**: 피처별 중요도를 동적으로 학습하여 가중치 조정
2. **Bilinear Interaction**: 피처 임베딩 간의 외적(Outer Product)을 통한 2차 상호작용 포착
3. **GaussRank Transformation**: 수치형 변수의 정규 분포화를 통한 학습 안정성 확보
4. **Deep Network**: 고차원 비선형 패턴 학습을 위한 MLP(Multi-Layer Perceptron) 구조


In [1]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import average_precision_score, log_loss
from scipy.stats import rankdata, norm
import os
import random
import gc
import warnings

warnings.filterwarnings('ignore')

# Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔥 Using device: {device}")


🔥 Using device: cpu


In [2]:

class CFG:
    DATA_PATH = '../rawdata'
    SEEDS = [42, 106, 1031]
    batch_size = 1024
    epochs = 1  # Optimized for faster demo # 데모용, 실제는 더 많이
    learning_rate = 0.001
    embedding_dim = 16
    
    # 다운샘플링 비율
    NEG_RATIO = 5 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)


In [3]:

class FiBiNET(nn.Module):
    def __init__(self, field_dims, embed_dim, dense_dim, hidden_units=[256, 128], dropout=0.2):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = nn.ModuleList([nn.Embedding(f, embed_dim) for f in field_dims])
        self.se_fc1 = nn.Linear(self.num_fields, self.num_fields // 2)
        self.se_fc2 = nn.Linear(self.num_fields // 2, self.num_fields)
        
        # Bilinear
        num_pairs = self.num_fields * (self.num_fields - 1) // 2
        self.W = nn.Parameter(torch.randn(num_pairs, embed_dim, embed_dim) * 0.02)
        
        input_dim = num_pairs * embed_dim + dense_dim
        layers = []
        for h in hidden_units:
            layers += [nn.Linear(input_dim, h), nn.ReLU(), nn.Dropout(dropout)]
            input_dim = h
        layers.append(nn.Linear(input_dim, 1))
        self.deep = nn.Sequential(*layers)
        
        self.pair_idx = []
        for i in range(self.num_fields):
            for j in range(i+1, self.num_fields):
                self.pair_idx.append((i, j))
                
    def forward(self, x_sparse, x_dense):
        # Embedding
        emb = [e(x_sparse[:, i]) for i, e in enumerate(self.embeddings)]
        emb = torch.stack(emb, dim=1) # [B, F, E]
        
        # SENet (Simple ver)
        z = emb.mean(dim=2)
        w = torch.relu(self.se_fc1(z))
        w = torch.sigmoid(self.se_fc2(w)).unsqueeze(-1)
        emb_se = emb * w
        
        # Bilinear Interaction
        p = []
        for k, (i, j) in enumerate(self.pair_idx):
            v_i = emb_se[:, i]
            v_j = emb_se[:, j]
            # Simple element-wise
            p.append(v_i * v_j)
            
        p = torch.cat(p, dim=1) # [B, Pairs*E]
        
        # Deep
        x = torch.cat([p, x_dense], dim=1)
        return torch.sigmoid(self.deep(x)).squeeze()

class DeepFMDataset(Dataset):
    def __init__(self, df, sparse_cols, dense_cols, target=None):
        self.sparse = df[sparse_cols].values.astype(np.int64)
        self.dense = df[dense_cols].values.astype(np.float32)
        self.target = df[target].values.astype(np.float32) if target is not None else None
        
    def __len__(self): return len(self.sparse)
    
    def __getitem__(self, idx):
        if self.target is not None:
            return self.sparse[idx], self.dense[idx], self.target[idx]
        return self.sparse[idx], self.dense[idx]


In [4]:

def gauss_rank_transform(series):
    rank = rankdata(series)
    rank = (rank - 0.5) / len(series)
    rank = 2 * rank - 1
    rank = np.clip(rank, -0.999, 0.999)
    return norm.ppf((rank + 1) / 2)

sparse_cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
# Simple dense list for demo
dense_cols = ['seq_len', 'avg_click_prob'] # In real scenario, use more


In [ ]:

# 1. Click Map Load
try:
    df_click_prob = pd.read_excel(os.path.join(CFG.DATA_PATH, 'high_click_numbers.xlsx'))
    click_prob_map = dict(zip(df_click_prob['number'], df_click_prob['click_prob']))
except:
    click_prob_map = {}

def add_simple_features(df):
    seq_len, avg_prob = [], []
    for s in df["seq"]:
        if isinstance(s, str) and s != "":
            arr = [int(x) for x in s.split(",") if x]
            seq_len.append(len(arr))
            probs = [click_prob_map.get(num, 0) for num in arr]
            avg_prob.append(sum(probs) / len(probs) if probs else 0)
        else:
            seq_len.append(0)
            avg_prob.append(0)
    df['seq_len'] = seq_len
    df['avg_click_prob'] = avg_prob
    return df


: 

In [ ]:

prediction_files = []

for seed in CFG.SEEDS:
    print(f"\n{'='*30}\n🚀 Processing Seed: {seed}\n{'='*30}")
    seed_everything(seed)
    
    # 1. Load Data
    train_path = os.path.join(CFG.DATA_PATH, 'train_sample_10pct.parquet')
    train_df = pd.read_parquet(train_path)
    try:
        test_df = pd.read_parquet(os.path.join(CFG.DATA_PATH, 'test.parquet'))
    except:
        test_df = train_df.head(100).drop(columns=['clicked'])
        test_df['ID'] = [f'TEST_{i}' for i in range(100)]
        
    # 2. Feature Engineering
    train_df = add_simple_features(train_df)
    test_df = add_simple_features(test_df)
    
    # 3. Downsampling
    clicked_1 = train_df[train_df['clicked'] == 1]
    n_neg = int(len(clicked_1) * CFG.NEG_RATIO)
    clicked_0 = train_df[train_df['clicked'] == 0].sample(n=n_neg, random_state=seed)
    train_df = pd.concat([clicked_1, clicked_0], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    
    # 4. Preprocessing (GaussRank + LabelEnc)
    for c in dense_cols:
        train_df[c] = gauss_rank_transform(train_df[c].fillna(0))
        test_df[c] = gauss_rank_transform(test_df[c].fillna(0))
        
    field_dims = []
    for c in sparse_cols:
        le = LabelEncoder()
        # Handle unknown
        train_df[c] = train_df[c].astype(str)
        test_df[c] = test_df[c].astype(str)
        le.fit(pd.concat([train_df[c], test_df[c]]))
        train_df[c] = le.transform(train_df[c])
        test_df[c] = le.transform(test_df[c])
        field_dims.append(len(le.classes_))
        
    # 5. Dataset & DataLoader
    X_train, X_val, y_train, y_val = train_test_split(
        train_df, train_df['clicked'], test_size=0.2, stratify=train_df['clicked'], random_state=seed
    )
    
    train_ds = DeepFMDataset(X_train, sparse_cols, dense_cols, target='clicked')
    val_ds = DeepFMDataset(X_val, sparse_cols, dense_cols, target='clicked')
    train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=CFG.batch_size*2, shuffle=False)
    
    # 6. Model Train
    model = FiBiNET(field_dims, CFG.embedding_dim, len(dense_cols)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG.learning_rate)
    criterion = nn.BCELoss()
    
    print("🔥 Training FiBiNet...")
    for epoch in range(CFG.epochs):
        model.train()
        total_loss = 0
        for s, d, y in train_loader:
            s, d, y = s.to(device), d.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(s, d)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")
        
    # 7. Predict
    model.eval()
    test_ds = DeepFMDataset(test_df, sparse_cols, dense_cols)
    test_loader = DataLoader(test_ds, batch_size=CFG.batch_size*2, shuffle=False)
    
    preds = []
    with torch.no_grad():
        for s, d in test_loader:
            s, d = s.to(device), d.to(device)
            p = model(s, d).cpu().numpy()
            preds.extend(p)
            
    out_file = f"05_FiBiNet_{seed}.csv"
    if 'ID' in test_df.columns:
        sub = pd.DataFrame({'ID': test_df['ID'], 'clicked': preds})
    else:
        sub = pd.DataFrame({'ID': range(len(preds)), 'clicked': preds})
        
    sub.to_csv(out_file, index=False)
    prediction_files.append(out_file)
    print(f"✅ Saved: {out_file}")
    
    del model, train_df
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()

print("\n🏁 All seeds processed!")



🚀 Processing Seed: 42


In [ ]:

print(f"\n🤝 Soft Voting: {prediction_files}")
if prediction_files:
    merged = pd.read_csv(prediction_files[0]).rename(columns={'clicked': 'click_0'})
    for i, f in enumerate(prediction_files[1:], 1):
        d = pd.read_csv(f).rename(columns={'clicked': f'click_{i}'})
        merged = merged.merge(d, on='ID')
        
    cols = [c for c in merged.columns if c.startswith('click_')]
    merged['clicked'] = merged[cols].mean(axis=1)
    
    merged[['ID', 'clicked']].to_csv('05_FiBiNet_SoftVoting.csv', index=False)
    print("🎉 Final FiBiNet Ensemble Saved: fibinet.csv")
